In [123]:
import numpy as np
import pandas as pd

data = pd.read_csv("data/unimelb_training.csv", header=0, dtype="str")
data = data.drop("Unnamed: 251", axis=1)
data["Year"] = data["Start.date"].map(lambda x: float(x.split("/")[2]))
data["Year"].dtype


numerical_data_feature_category = [
    ("RFCD.Percentage.", 5), 
    ("SEO.Percentage.", 5), 
    ("Year.of.Birth.", 15),
    ("Number.of.Successful.Grant.", 15),
    ("Number.of.Unsuccessful.Grant.", 15),
    ("A..", 15),
    ("A.", 15),
    ("B.", 15),
    ("C.", 15)
]

for feature_name, feature_range in numerical_data_feature_category:
    numerical_data_features = [feature_name+str(i) for i in range(1, feature_range+1)]
    for numerical_data_feature in numerical_data_features:
        data[numerical_data_feature] = data[numerical_data_feature].astype(float)

data.dtypes

Grant.Application.ID                        object
Grant.Status                                object
Sponsor.Code                                object
Grant.Category.Code                         object
Contract.Value.Band...see.note.A            object
Start.date                                  object
RFCD.Code.1                                 object
RFCD.Percentage.1                          float64
RFCD.Code.2                                 object
RFCD.Percentage.2                          float64
RFCD.Code.3                                 object
RFCD.Percentage.3                          float64
RFCD.Code.4                                 object
RFCD.Percentage.4                          float64
RFCD.Code.5                                 object
RFCD.Percentage.5                          float64
SEO.Code.1                                  object
SEO.Percentage.1                           float64
SEO.Code.2                                  object
SEO.Percentage.2               

In [124]:
data.head()

,Grant.Application.ID,Grant.Status,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,Start.date,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,...,Faculty.No..15,With.PHD.15,No..of.Years.in.Uni.at.Time.of.Grant.15,Number.of.Successful.Grant.15,Number.of.Unsuccessful.Grant.15,A..15,A.15,B.15,C.15,Year
0,1,1,NaN,NaN,A,8/11/05,280199,100.0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1,2,1,2B,10A,B,11/11/05,280103,30.0,280106,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2,3,1,29A,10B,A,14/11/05,321004,60.0,321216,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,4,1,40D,10B,C,15/11/05,270602,50.0,320602,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
4,5,0,59C,10A,A,16/11/05,260500,34.0,280000,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


# Contigency for persons

In [125]:
# build the list of all unique values of a given spread column, as a tuple
def process_column(data, prefix, nb_max):
    unique_values = set([])
    for i in range(1, nb_max+1):
        unique_values.update(data[prefix+str(i)].unique())
    return tuple(unique_values)

# Build a dataframe with contingecy frequencies from a given spread column
def add_contigency(prefix, nb_max, unique_values, add_cardinal, keep_nan):
    nb_unique_values = len(unique_values)
    has_nans = np.nan in unique_values
    nan_index = unique_values.index(np.nan)
    new_feature_labels = [prefix+str(value) for value in unique_values]
    if has_nans and not keep_nan:
        new_feature_labels.pop(nan_index)
    if add_cardinal:
        new_feature_labels.append("cardinal_"+prefix)
    def nested_func(x):
        values_frequency = [0]*nb_unique_values
        for i in range(1, nb_max+1):
            values_frequency[unique_values.index(x[prefix+str(i)])] += 1
        if has_nans:
            values_frequency[nan_index] = 0
            if not keep_nan:
                values_frequency.pop(nan_index)
        summe = sum(values_frequency)
        if summe != 0:
            values_frequency = [value/summe for value in values_frequency]
        if add_cardinal:
            values_frequency.append(summe)
        return pd.Series(values_frequency, index=new_feature_labels)
    return nested_func

# Build a dataframe with contingecy frequencies from a given spread column
def build_contigency(data, prefix, nb_max, add_cardinal=False, keep_nan=False):
    unique_values = process_column(data, prefix, nb_max)
    return unique_values, data.apply(add_contigency(prefix, nb_max, unique_values, add_cardinal, keep_nan), axis=1, raw=True)

In [126]:
contigency_features_names = []

## Role

In [127]:
unique_roles, role_contigency_df = build_contigency(data, "Role.", 15, add_cardinal=True)

print(unique_roles)
contigency_features_names.extend(list(role_contigency_df.columns))
data = data.join(role_contigency_df)

(nan, 'HONVISIT', 'EXT_CHIEF_INVESTIGATOR', 'CHIEF_INVESTIGATOR', 'STUDRES', 'STUD_CHIEF_INVESTIGATOR', 'DELEGATED_RESEARCHER', 'EXTERNAL_ADVISOR', 'PRINCIPAL_SUPERVISOR')


## Number of years at uni at time of grant

In [128]:
# No..of.Years.in.Uni.at.Time.of.Grant.

unique_yearsinuni, yearsinuni_contigency_df = build_contigency(data, "No..of.Years.in.Uni.at.Time.of.Grant.", 15)

print(unique_yearsinuni)
contigency_features_names.extend(list(yearsinuni_contigency_df.columns))
data = data.join(yearsinuni_contigency_df)

('>=0 to 5', nan, '>5 to 10', '>10 to 15', 'more than 15', 'Less than 0')


## Country of birth

In [129]:
# Country.of.Birth.

unique_cob, cob_contigency_df = build_contigency(data, "Country.of.Birth.", 15)

print(unique_cob)
contigency_features_names.extend(list(cob_contigency_df.columns))
data = data.join(cob_contigency_df)

(nan, 'Western Europe', 'South Africa', 'Great Britain', 'New Zealand', 'The Americas', 'Australia', 'Eastern Europe', 'North America', 'Middle East and Africa', 'Asia Pacific')


# aggregations for persons

In [130]:
# Build a dataframe with contingecy frequencies from a given spread column
def add_aggregation(prefixes, nb_max):
    feature_names_by_prefix = [[prefix+str(i) for i in range(1, nb_max+1)] for prefix in prefixes]
    methods_to_apply = [np.mean]
    new_feature_labels_by_prefix = [["mean_"+prefix] for prefix in prefixes]
    flat_new_labels = [new_feature_label for new_feature_labels in new_feature_labels_by_prefix for new_feature_label in new_feature_labels]
    def nested_func(x):
        values_by_prefix = [
            [x[feature_name] for feature_name in feature_names if np.isreal(x[feature_name]) and not np.isnan(x[feature_name])]
            for feature_names in feature_names_by_prefix
        ]
        aggregated_values_by_prefix = [[method(values) for method in methods_to_apply] if len(values) > 0 else [np.nan]*len(methods_to_apply) for values in values_by_prefix]
        return pd.Series(
            data=[aggregated_value for aggregated_values in aggregated_values_by_prefix for aggregated_value in aggregated_values], 
            index=flat_new_labels
        )
    return nested_func

# Build a dataframe with contingecy frequencies from a given spread column
def build_aggregation(data, prefix, nb_max):
    return data.apply(add_aggregation(prefix, nb_max), axis=1, raw=True)

## Year of birth / Number of successful grants / Number of unsuccessfull grants / journals publications (A*, A, B, C)

In [131]:
aggregation_df = build_aggregation(data, ["Year.of.Birth.", "Number.of.Successful.Grant.", "Number.of.Unsuccessful.Grant.", "A..", "A.", "B.", "C."], 15)
aggregation_features_names = list(aggregation_df.columns)

print(aggregation_features_names)
data = data.join(aggregation_df)

['mean_Year.of.Birth.', 'mean_Number.of.Successful.Grant.', 'mean_Number.of.Unsuccessful.Grant.', 'mean_A..', 'mean_A.', 'mean_B.', 'mean_C.']


# RFCD & SEO

## RFCD

In [132]:
def rfcd_codes(data, prefix, nb_max, length):
    unique_values = set([])
    for i in range(1, nb_max+1):
        unique_values.update([str(code)[:length] if not np.isreal(code) else np.nan for code in data[prefix+str(i)].unique()])
    return tuple(unique_values)

# rcfd_codes_unique = rfcd_codes(data, "RFCD.Code.")

# Build a dataframe with contingecy frequencies from a given spread column
def add_rfcd_aggregation(prefix1, prefix2, nb_max, unique_values, add_cardinal, suffix, length, keep_nan):
    nb_unique_values = len(unique_values)
    has_nans = np.nan in unique_values
    nan_index = unique_values.index(np.nan)
    new_feature_labels = [prefix2+str(value)+suffix for value in unique_values]
    if has_nans and not keep_nan:
        new_feature_labels.pop(nan_index)
    if add_cardinal:
        new_feature_labels.append("cardinal_"+prefix2)
    def nested_func(x):
        values_percentage = [0]*nb_unique_values
        cardinal = 0
        for i in range(1, nb_max+1):
            considered_value = x[prefix1+str(i)]
            current_value =  np.nan
            if not np.isreal(considered_value) and considered_value != '0':
                current_value = str(considered_value)[:length]
                cardinal += 1
            values_percentage[unique_values.index(current_value)] += x[prefix2+str(i)]/100.0
        if has_nans:
            values_percentage[nan_index] = 0
            if not keep_nan:
                values_percentage.pop(nan_index)
        if add_cardinal:
            values_percentage.append(cardinal)
        return pd.Series(values_percentage, index=new_feature_labels)
    return nested_func

# Build a dataframe with contingecy frequencies from a given spread column
def build_rfcd_aggregation(data, prefix1, prefix2, nb_max, length, add_cardinal=True, suffix="x", keep_nan=False):
    unique_values = rfcd_codes(data, prefix1, nb_max, length)
    return unique_values, data.apply(add_rfcd_aggregation(prefix1, prefix2, nb_max, unique_values, add_cardinal, suffix, length, keep_nan), axis=1, raw=True)

In [133]:
rfcd_codes_unique, rfcd_df = build_rfcd_aggregation(data, "RFCD.Code.", "RFCD.Percentage.", 5, 1)

rfcd_features_names = list(rfcd_df.columns)
print(rfcd_codes_unique)
print(rfcd_features_names)
data = data.join(rfcd_df)

(nan, '0', '3', '2', '9', '4')
['RFCD.Percentage.0x', 'RFCD.Percentage.3x', 'RFCD.Percentage.2x', 'RFCD.Percentage.9x', 'RFCD.Percentage.4x', 'cardinal_RFCD.Percentage.']


## SEO

In [134]:
seo_codes_unique, seo_df = build_rfcd_aggregation(data, "SEO.Code.", "SEO.Percentage.", 5, 1)

seo_features_names = list(seo_df.columns)
print(seo_codes_unique)
print(seo_features_names)
data = data.join(seo_df)

(nan, '9', '7', '6', '0')
['SEO.Percentage.9x', 'SEO.Percentage.7x', 'SEO.Percentage.6x', 'SEO.Percentage.0x', 'cardinal_SEO.Percentage.']


In [135]:
data.head(20)

,Grant.Application.ID,Grant.Status,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,Start.date,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,...,RFCD.Percentage.3x,RFCD.Percentage.2x,RFCD.Percentage.9x,RFCD.Percentage.4x,cardinal_RFCD.Percentage.,SEO.Percentage.9x,SEO.Percentage.7x,SEO.Percentage.6x,SEO.Percentage.0x,cardinal_SEO.Percentage.
0,1,1,NaN,NaN,A,8/11/05,280199,100.0,0,0.0,...,0.00,1.00,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,2,1,2B,10A,B,11/11/05,280103,30.0,280106,30.0,...,0.00,1.00,0.0,0.0,3.0,0.0,1.0,0.0,0.0,2.0
2,3,1,29A,10B,A,14/11/05,321004,60.0,321216,40.0,...,1.00,0.00,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0
3,4,1,40D,10B,C,15/11/05,270602,50.0,320602,50.0,...,0.50,0.50,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0
4,5,0,59C,10A,A,16/11/05,260500,34.0,280000,33.0,...,0.00,1.00,0.0,0.0,3.0,0.0,1.0,0.0,0.0,1.0
5,6,1,4D,10A,NaN,19/11/05,321204,100.0,0,0.0,...,1.00,0.00,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
6,7,0,2B,10A,NaN,19/11/05,270708,50.0,270203,30.0,...,0.00,1.00,0.0,0.0,3.0,0.0,1.0,0.0,0.0,2.0
7,8,0,28D,30B,A,19/11/05,321405,100.0,0,0.0,...,1.00,0.00,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
8,9,1,2B,10A,H,19/11/05,260108,50.0,260109,50.0,...,0.00,1.00,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0
9,10,1,2B,10A,NaN,19/11/05,270708,40.0,270704,30.0,...,0.00,1.00,0.0,0.0,3.0,0.0,1.0,0.0,0.0,2.0


# Building dataset

In [136]:
application_feature_names = [
    "Grant.Application.ID", "Grant.Status", "Sponsor.Code", "Grant.Category.Code", "Contract.Value.Band...see.note.A", "Start.date"
]

application_feature_names + contigency_features_names + aggregation_features_names + rfcd_features_names + seo_features_names

['Grant.Application.ID',
 'Grant.Status',
 'Sponsor.Code',
 'Grant.Category.Code',
 'Contract.Value.Band...see.note.A',
 'Start.date',
 'Role.HONVISIT',
 'Role.EXT_CHIEF_INVESTIGATOR',
 'Role.CHIEF_INVESTIGATOR',
 'Role.STUDRES',
 'Role.STUD_CHIEF_INVESTIGATOR',
 'Role.DELEGATED_RESEARCHER',
 'Role.EXTERNAL_ADVISOR',
 'Role.PRINCIPAL_SUPERVISOR',
 'cardinal_Role.',
 'No..of.Years.in.Uni.at.Time.of.Grant.>=0 to 5',
 'No..of.Years.in.Uni.at.Time.of.Grant.>5 to 10',
 'No..of.Years.in.Uni.at.Time.of.Grant.>10 to 15',
 'No..of.Years.in.Uni.at.Time.of.Grant.more than 15',
 'No..of.Years.in.Uni.at.Time.of.Grant.Less than 0',
 'Country.of.Birth.Western Europe',
 'Country.of.Birth.South Africa',
 'Country.of.Birth.Great Britain',
 'Country.of.Birth.New Zealand',
 'Country.of.Birth.The Americas',
 'Country.of.Birth.Australia',
 'Country.of.Birth.Eastern Europe',
 'Country.of.Birth.North America',
 'Country.of.Birth.Middle East and Africa',
 'Country.of.Birth.Asia Pacific',
 'mean_Year.of.Birth.'

In [137]:
data[application_feature_names + contigency_features_names + aggregation_features_names + rfcd_features_names + seo_features_names].isnull().sum()

Grant.Application.ID                                    0
Grant.Status                                            0
Sponsor.Code                                          912
Grant.Category.Code                                   912
Contract.Value.Band...see.note.A                     3563
Start.date                                              0
Role.HONVISIT                                           0
Role.EXT_CHIEF_INVESTIGATOR                             0
Role.CHIEF_INVESTIGATOR                                 0
Role.STUDRES                                            0
Role.STUD_CHIEF_INVESTIGATOR                            0
Role.DELEGATED_RESEARCHER                               0
Role.EXTERNAL_ADVISOR                                   0
Role.PRINCIPAL_SUPERVISOR                               0
cardinal_Role.                                          0
No..of.Years.in.Uni.at.Time.of.Grant.>=0 to 5           0
No..of.Years.in.Uni.at.Time.of.Grant.>5 to 10           0
No..of.Years.i

# Setting train and test sets

In [138]:
testing_ids = pd.read_csv("data/testing_ids.csv")
len(testing_ids)

518

In [183]:
trainin2_ids = pd.read_csv("data/training2_ids.csv")
len(trainin2_ids)

1557

In [204]:
# filling NAs with mean
for feature_name in ["mean_Year.of.Birth.", "mean_Number.of.Successful.Grant.", "mean_Number.of.Unsuccessful.Grant.", "mean_A..", "mean_A.", "mean_B.", "mean_C."]:
    data[feature_name] = data[feature_name].fillna(np.nanmean(data[feature_name]))

# filling NAs with other value
# data["Sponsor.Code_no_nas"] = data["Sponsor.Code"].fillna("Unknown")
# len(data["Sponsor.Code_no_nas"].astype("category").cat.codes.unique())

In [205]:
features_without_nan = list(np.concatenate([application_feature_names + contigency_features_names + aggregation_features_names + rfcd_features_names + seo_features_names]))
features_without_nan.remove("Sponsor.Code")
features_without_nan.remove("Grant.Category.Code")
features_without_nan.remove("Contract.Value.Band...see.note.A")
# features_without_nan.remove("mean_Year.of.Birth.")
# features_without_nan.remove("mean_Number.of.Successful.Grant.")
# features_without_nan.remove("mean_Number.of.Unsuccessful.Grant.")
# features_without_nan.remove("mean_A..")
# features_without_nan.remove("mean_A.")
# features_without_nan.remove("mean_B.")
# features_without_nan.remove("mean_C.")

# Not to be used for classification
features_without_nan.remove("Grant.Application.ID")
features_without_nan.remove("Grant.Status")
features_without_nan.remove("Start.date")

data[features_without_nan].isnull().sum()

Role.HONVISIT                                        0
Role.EXT_CHIEF_INVESTIGATOR                          0
Role.CHIEF_INVESTIGATOR                              0
Role.STUDRES                                         0
Role.STUD_CHIEF_INVESTIGATOR                         0
Role.DELEGATED_RESEARCHER                            0
Role.EXTERNAL_ADVISOR                                0
Role.PRINCIPAL_SUPERVISOR                            0
cardinal_Role.                                       0
No..of.Years.in.Uni.at.Time.of.Grant.>=0 to 5        0
No..of.Years.in.Uni.at.Time.of.Grant.>5 to 10        0
No..of.Years.in.Uni.at.Time.of.Grant.>10 to 15       0
No..of.Years.in.Uni.at.Time.of.Grant.more than 15    0
No..of.Years.in.Uni.at.Time.of.Grant.Less than 0     0
Country.of.Birth.Western Europe                      0
Country.of.Birth.South Africa                        0
Country.of.Birth.Great Britain                       0
Country.of.Birth.New Zealand                         0
Country.of

In [206]:
# Setting index
#data = data.reset_index()
training_ids = np.setdiff1d(data["Grant.Application.ID"].astype(int).values, testing_ids["ids"].values)
#data = data.set_index("Grant.Application.ID")

test_df = data.loc[testing_ids["ids"].values-1]
train_df = data.loc[training_ids-1]

In [207]:
X_train, X_test = train_df[features_without_nan], test_df[features_without_nan]
y_train, y_test = train_df["Grant.Status"], test_df["Grant.Status"]

In [212]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Predictions
classifier = LogisticRegression().fit(X_train, y_train)
#classifier = RandomForestClassifier(n_estimators=500).fit(X_train, y_train)
prediction = classifier.predict(X_test)

accuracy_score(y_test, prediction)

0.73166023166023164

In [166]:
X_train

,Role.HONVISIT,Role.EXT_CHIEF_INVESTIGATOR,Role.CHIEF_INVESTIGATOR,Role.STUDRES,Role.STUD_CHIEF_INVESTIGATOR,Role.DELEGATED_RESEARCHER,Role.EXTERNAL_ADVISOR,Role.PRINCIPAL_SUPERVISOR,cardinal_Role.,No..of.Years.in.Uni.at.Time.of.Grant.>=0 to 5,...,RFCD.Percentage.3x,RFCD.Percentage.2x,RFCD.Percentage.9x,RFCD.Percentage.4x,cardinal_RFCD.Percentage.,SEO.Percentage.9x,SEO.Percentage.7x,SEO.Percentage.6x,SEO.Percentage.0x,cardinal_SEO.Percentage.
0,0.0,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.000000,1.0,0.000000,...,0.00,1.00,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.000000,1.0,0.000000,...,0.00,1.00,0.0,0.0,3.0,0.0,1.0,0.0,0.0,2.0
2,0.0,0.428571,0.428571,0.0,0.000000,0.142857,0.0,0.000000,7.0,0.000000,...,1.00,0.00,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0
3,0.0,0.500000,0.166667,0.0,0.166667,0.000000,0.0,0.166667,6.0,0.500000,...,0.50,0.50,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0
4,0.0,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.000000,1.0,0.000000,...,0.00,1.00,0.0,0.0,3.0,0.0,1.0,0.0,0.0,1.0
5,0.0,0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.000000,1.0,1.000000,...,1.00,0.00,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
6,0.0,0.250000,0.750000,0.0,0.000000,0.000000,0.0,0.000000,4.0,0.000000,...,0.00,1.00,0.0,0.0,3.0,0.0,1.0,0.0,0.0,2.0
7,0.0,0.333333,0.666667,0.0,0.000000,0.000000,0.0,0.000000,3.0,0.000000,...,1.00,0.00,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
8,0.0,0.500000,0.500000,0.0,0.000000,0.000000,0.0,0.000000,4.0,0.000000,...,0.00,1.00,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0
9,0.0,0.750000,0.250000,0.0,0.000000,0.000000,0.0,0.000000,4.0,0.000000,...,0.00,1.00,0.0,0.0,3.0,0.0,1.0,0.0,0.0,2.0


In [182]:
data["mean_Year.of.Birth."].isnull()

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
8678    False
8679    False
8680    False
8681    False
8682    False
8683    False
8684    False
8685    False
8686    False
8687    False
8688    False
8689    False
8690    False
8691    False
8692    False
8693    False
8694    False
8695    False
8696    False
8697    False
8698    False
8699    False
8700    False
8701    False
8702    False
8703    False
8704    False
8705    False
8706    False
8707    False
Name: mean_Year.of.Birth., dtype: bool